# Extracción de datos

#### Supuetos del ejercicio: Tenemos una base de datos de contenido en netflix en una base de datos SQL, y queremos hacer un pipeline sencillo donde se extraiga la base y se cargue a la base de datos una tabla por cada director con su información correspondiente y otra donde tabla de la información donde no se conoce al director.

In [1]:
# Descargamos librerías y obtenemos el dataset de Netflix
import sqlite3 
import pandas as pd
import kagglehub
import numpy as np 

# Download latest version
path = kagglehub.dataset_download("shivamb/netflix-shows")

print("Path to dataset files:", path)

C:\Users\brand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\brand\.cache\kagglehub\datasets\shivamb\netflix-shows\versions\5


In [2]:
#Conexión a la base de datos
def sql_connection(mydata):
    try:
        con = sqlite3.connect(mydata)
        return con
    except sqlite3.Error:
        print("The connection failed")

con = sql_connection("C:/DB/netflixDB.db")
cursorDB=con.cursor()


In [3]:
# Extracción y visualización de los datos
datapd = pd.read_sql_query("SELECT * FROM netflix_titles", con)
datapd.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


# Imputación 

In [4]:
# Analizamos valores nulos
nul_cols = datapd.isnull().sum()
print(nul_cols)

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [5]:
# Imputamos los valores nulos con numpy
# Principalmente porque numpy está optimizado y junto con pandas tienen mejor tratamiento de los valores nulos
datapd.fillna(value=np.nan,inplace=True)

#Revisamos la lista de Directores
direct=datapd['director'].unique()

# Cargar Data

In [ ]:
for i in range(len(direct)):
    if type(direct[i]) != float:
        direct_data = datapd[datapd['director'] == direct[i]]
        direct_data.to_sql(direct[i], con, if_exists='replace', index=False)
    else:
        datapd['director'] = datapd['director'].replace(np.nan, 'No director')
        direct_data = datapd[datapd['director'] == 'No director']
        direct_data.to_sql('No director', con, if_exists='replace', index=False)
con.close()